The goal of this notebook is to find optimal combinations of parameters for minimum model error. 

In [ ]:
import numpy as np
import scipy as sp
from scipy.integrate import odeint
from scipy import stats
from scipy import interpolate
import math
import matplotlib.pyplot as plt
import pandas as pd # to read excel
# print(f"pandas version is {pd.__version__}")
import seaborn as sns
from array import *
import researchpy as rp
import scipy.stats as stats
%matplotlib inline

from local_functions import Time_to_Hours, plot_result_extra

import datetime

In [ ]:
# 1. Indoor system
#   a. Read data and parameters from excel file

xl = pd.ExcelFile('../data/Parameters.xlsx')
x2 = pd.ExcelFile('../data/Data.xlsx')

# Import parameters into df1
df1 = xl.parse('Parameters')

# assigmment of parameters to values: 
for key,val in zip(df1.Parameter,df1.Indoor_value):
    exec(key + '=val')
    print(key,val)

# Import indoor data into df2
df2 = x2.parse('Indoor')

df2.head()

In [ ]:
# b. process data (indoor measurements)

    # 1. Convert Time into hours
T = []
for i in df2['Sample'][:]:
    T1 = df2['Timei'][df2['Sample'].values.tolist().index(i)]
    T2 = df2['Time'][df2['Sample'].values.tolist().index(i)]
    T.append(Time_to_Hours(T1,T2))
df2['T'] = T

    # 2. Make a temp DF (df2_temp) by filtering the original df - remove 'wierd' results (i.e sporulation) 

df2_Reduced = df2[(df2.Treatment != 'Acclimation') & (df2.Week <= 3) & (df2.Stage != 'i') & (df2.Sporulated == 'No')&(df2.Comment != 'Exclude')]
df2_Reduced

In [ ]:
df2_ReducedTemp = df2_Reduced[(df2_Reduced.Exp != 1)&(df2_Reduced.Exp != 2)&(df2_Reduced.Exp != 5)]
df2_ReducedTemp

In [ ]:
# 4. Calculate model results for:
#   a. Indoor system
    # I. Produce an array ("m/Nint/Next_mod_all") of all model results (hours 1-504) for each treatment

In [ ]:
# optimized parameters: bounds
#'µmax'
problem = {
    'num_vars': 9,
    'names': ['λ20','Nintmax','Nintcrit','Nextlosses','Ks','Vmax','KI','K0','Ka'],
    'bounds': [[0.001,0.005],
               [4.5,5],
               [0.7,3.2],
               [0.001,0.02],
               [10,30],
               [50,250],
               [15, 150],
               [0.1,3],
               [0.01,0.2]]
}

print(problem)

In [ ]:
problem['bounds']

In [ ]:
from scipy.optimize import Bounds
tmp = np.array(problem['bounds']).T.squeeze().tolist()
bounds = Bounds(tmp[0],tmp[1])

In [ ]:
def controlled_N_constST(x,t,Nintmax,Nintmin,Vmax,Ks,dNextoutdt,dNextindt,miu,dmoutdt,Nintcrit,Z,KI,K0,Ka,
                losses20,teta,umol_to_percent_DW,I0,Temp):


    Next = x[0] # units: [umol N/l]
    Nint = x[1] # units: [% g N/g DW]
    m = x[2] # units: [g DW/l]
    
    Neff = (Nintmax - Nint)/(Nintmax - Nintmin)  # units: [ ]
    if Next <= 0:
        uN = 0
    else: 
        uN = Vmax * Next / (Ks + Next)  # units: [umol N/g DW/h]
    if Nint >= Nintcrit:
        fN = 1
    else:
        fN = ((Nint - Nintmin)/Nint) / ((Nintcrit - Nintmin)/Nintcrit)  # units: [ ]
    fP = 1  #(N:P < 12)


    # density - light penetration effects:
    SD = m * 5 / (0.2 * 0.178) # Stocking Density. units: [g DW/ m^2]
    I_average = (I0(t) / (K0 * Z + Ka * SD)) * (1 - np.exp(-(K0 * Z + Ka * SD))) # units: [umul photons/(m^2 s)]
    fI = I_average / (I_average + KI) # units: [-]
    # print(f"t={t} I0={I0(t)} fI={fI}")
    
        
    
    # empirically defined losses
    losses = losses20 * teta ** (Temp - 20)

    # limiting factors:
    g = min(fN,fI,fP)
    # print(f"t={t} g= {g} fN = {fN}, fI = {fI} fP = {fP} fT = {fT} fS = {fS}")

    
    umol_to_percent_DW = 100*14e-6 #[% g N/umol N] 
    
    # Reactor Next -> Nint -> m and feedback

    dNextdt = - Neff * uN * m  - dNextoutdt * Next + dNextindt # [umol N/l/h] # added *Next
    dNintdt = umol_to_percent_DW * Neff * uN - Nint * miu * g # units: [%g N/g DW/h]

    if fI == 0:
        losses = 0
    else: 
        losses = losses20 * teta ** (Temp - 20)
    dmdt = (miu * g - losses) * m #units: [g DW/l/h]
    
    #dNextdt = -Neff * uN * m - dNextoutdt + dNextindt # units: [umol N/l/h]
    #dNintdt = umol_to_percent_DW * Neff * uN - Nint * miu * fN  #units: [%g N/g DW/h]
    #dmdt = miu * fN * m - dmoutdt #units: [g DW/l/h]
    
    return [dNextdt,dNintdt,dmdt]

In [ ]:
# model simulations with different parameteric combinations


Temp = 22
S = 39 # fix salinity function and S=40

Treatments = ['1000/1/168','500/2/168','500/3/168','2000/1/168','200/5/168']
Nint0All = ['2.12','2.13','2.32','2.05','1.3']

def residual(X, plot_on = False):
    """ estimates the residual as total RMS for the given set of parameters X """
   
    #miu = X[0]
    lossess20 = X[0]
    Nintmax = X[1]
    Nintcrit = X[2]
    dNextoutdt = X[3]
    Ks = X[4]
    Vmax = X[5]
    KI = X[6]
    K0 = X[7]
    Ka = X[8]


    if plot_on:
        df2_Reduced = df2[(df2.Treatment != 'Acclimation') & (df2.Week <= 3) & (df2.Stage != 'i') & (df2.Sporulated == 'No')&(df2.Comment != 'Exclude')]
        df2_cal = df2_Reduced[(df2_Reduced.Exp != 1)&(df2_Reduced.Exp != 2)&(df2_Reduced.Exp != 5)]
        df2_val = df2_Reduced[(np.isnan(df2_Reduced.DW) != True)&(df2_Reduced.Exp != 3)&(df2_Reduced.Exp != 4)]
        df2_spor = df2[(np.isnan(df2.DW) != True)&(df2.Treatment != 'Acclimation')&(df2.Duration == 168) & (df2.Week <= 3) & (df2.Stage != 'i') & (df2.Sporulated == 'Yes')&(df2.Comment != 'Exclude')]

    
    TModAll = [] # Each sub-array has the time steps of a specific treatment (periods)
    mExpAllTimes,NintExpAllTimes,NextExpAllTimes = [],[],[]
    mModAll,NintModAll,NextModAll = [],[],[]

    for i in Treatments:
        
        df2Temp = df2_ReducedTemp[(df2_ReducedTemp.Treatment == i)]
        mTemp = df2Temp[(~np.isnan(df2Temp.DW))]['DW']
        mTimeTemp = df2Temp[(~np.isnan(df2Temp.DW))]['T']
        NintTemp = df2Temp[(~np.isnan(df2Temp.N))]['N']
        NintTimeTemp = df2Temp[(~np.isnan(df2Temp.N))]['T']
        NextTemp = df2Temp[(~np.isnan(df2Temp.NH4))]['NH4']
        NextTimeTemp = df2Temp[(~np.isnan(df2Temp.NH4))]['T']

        if plot_on:
            df2Temp_cal = df2_cal[(df2_cal.Treatment == i)]
            mTemp_cal = df2Temp_cal[(np.isnan(df2Temp_cal.DW) != True)]['DW']
            mTimeTemp_cal = df2Temp_cal[(np.isnan(df2Temp_cal.DW) != True)]['T']
            NintTemp_cal = df2Temp_cal[(np.isnan(df2Temp_cal.N) != True)]['N']
            NintTimeTemp_cal = df2Temp_cal[(np.isnan(df2Temp_cal.N) != True)]['T']
            NextTemp_cal = df2Temp_cal[(np.isnan(df2Temp_cal.NH4) != True)]['NH4']
            NextTimeTemp_cal = df2Temp_cal[(np.isnan(df2Temp_cal.NH4) != True)]['T']
            
            # validation data
            df2Temp_val = df2_val[(df2_val.Treatment == i)]
            mTemp_val = df2Temp_val[(np.isnan(df2Temp_val.DW) != True)]['DW']
            mTimeTemp_val = df2Temp_val[(np.isnan(df2Temp_val.DW) != True)]['T']
            NintTemp_val = df2Temp_val[(np.isnan(df2Temp_val.N) != True)]['N']
            NintTimeTemp_val = df2Temp_val[(np.isnan(df2Temp_val.N) != True)]['T']
            NextTemp_val = df2Temp_val[(np.isnan(df2Temp_val.NH4) != True)]['NH4']
            NextTimeTemp_val = df2Temp_val[(np.isnan(df2Temp_val.NH4) != True)]['T']
            mTemp_val = df2Temp_val[(~np.isnan(df2Temp_val.DW))]['DW']
            mTimeTemp_val = df2Temp_val[(~np.isnan(df2Temp_val.DW))]['T']
            NintTemp_val = df2Temp_val[(~np.isnan(df2Temp_val.N))]['N']
            NintTimeTemp_val = df2Temp_val[(~np.isnan(df2Temp_val.N))]['T']
            NextTemp_val = df2Temp_val[(~np.isnan(df2Temp_val.NH4))]['NH4']
            NextTimeTemp_val = df2Temp_val[(~np.isnan(df2Temp_val.NH4))]['T']
            
            # sporulation data
            df2Temp_spor = df2_spor[(df2_spor.Treatment == i)]
            mTemp_spor = df2Temp_spor[(np.isnan(df2Temp_spor.DW) != True)]['DW']
            mTimeTemp_spor = df2Temp_spor[(np.isnan(df2Temp_spor.DW) != True)]['T']
            NintTemp_spor = df2Temp_spor[(np.isnan(df2Temp_spor.N) != True)]['N']
            NintTimeTemp_spor = df2Temp_spor[(np.isnan(df2Temp_spor.N) != True)]['T']
            NextTemp_spor = df2Temp_spor[(np.isnan(df2Temp_spor.NH4) != True)]['NH4']
            NextTimeTemp_spor = df2Temp_spor[(np.isnan(df2Temp_spor.NH4) != True)]['T']
            mTemp_spor = df2Temp_spor[(~np.isnan(df2Temp_spor.DW))]['DW']
            mTimeTemp_spor = df2Temp_spor[(~np.isnan(df2Temp_spor.DW))]['T']
            NintTemp_spor = df2Temp_spor[(~np.isnan(df2Temp_spor.N))]['N']
            NintTimeTemp_spor = df2Temp_spor[(~np.isnan(df2Temp_spor.N))]['T']
            NextTemp_spor = df2Temp_spor[(~np.isnan(df2Temp_spor.NH4))]['NH4']
            NextTimeTemp_spor = df2Temp_spor[(~np.isnan(df2Temp_spor.NH4))]['T']

            print('Number of samples:\nm: ' + str(len(mTimeTemp_cal)) + ' Nint: ' + str(len(NintTimeTemp_cal)) + ' Next: ' + str(len(NextTimeTemp_cal)))



        Tr = []
        Tr = i.split('/')
        Tr = [float(i) for i in Tr]
        Amplitude = Tr[0]
        Period = float(7/Tr[1]) # Period is in hours
        Duration = Tr[2]/24

        NEXT, NINT, M, TT = [],[],[],[]

        n_days = Duration*3
        count_periods = 0

        # hour = 0 is 1pm
        # in the indoor settings we have to solve for shorter periods, 
        # at the beginning of the Treatment we reset Nint, Next, m
        # at the beginning of each Period we set Next to the last Amplitude, 
        # m0 to the end of previous solution of ode()

        # Let's prepare the IO(t) function that will be supplied to odeint 
        # instead of a scalar. 

        all_treatment_hours = np.arange(0,n_days*24,dtype=int)

        # 8pm on the first day is zero crossing of this one 8pm - 10m = 7 hours
        offtimes = all_treatment_hours[np.mod(all_treatment_hours - 7,24) == 0]
        offtimes = np.r_[offtimes,all_treatment_hours[-1]+1] # last hour would be whatever

        # 6am on the next day is zero crossing of this one, 6am - 8pm is 10 hours
        ontimes = all_treatment_hours[np.mod(all_treatment_hours - 7 + 10,24) == 0]
        # and the iniital hour as we always start at 1pm, during ontime
        ontimes = np.r_[int(0),ontimes]

        # prepare the duty cycle
        I0set = np.zeros_like(all_treatment_hours)
        for s,e in zip(ontimes,offtimes):
            I0set[s:e] = 80

        # if you want to replace it by a "constant" I0 then replace the lines above with
        # the following line and then it will also give you a constant solution you had before

        # I0set = np.ones_like(all_treatment_hours)*80


        I0 = interpolate.interp1d(all_treatment_hours, I0set, bounds_error=False, fill_value="extrapolate")

        for hour in np.arange(0,n_days*24,round(Period*24,0)):
            if plot_on:
                print(f'Hour = {hour}')
            if hour == 0:
                if plot_on:
                    print('Starting point')
                Nint_0 = Nint0All[Treatments.index(i)]
                m_0 = m0
                Next_0 = Amplitude

            if hour > 0 and np.mod(hour,round(Period*24,0)) == 0:
                count_periods = count_periods + 1

                if count_periods == Tr[1]:
                    if plot_on:
                        print('Duration')
                    # reset everything, except Nint
                    Nint_0 = NINT[-1][-1]
                    Next_0 = Amplitude
                    m_0 = m0
                    count_periods = 0
                else:
                    # period passed, not Duration
                    # add amplitude, keep going
                    if plot_on:
                        print('Period') 
                    Next_0 = NEXT[-1][-1] + Amplitude
                    Nint_0 = NINT[-1][-1]
                    m_0 = M[-1][-1]


            # Here we want to send odeint the times of the light sub-period or 
            # darkness sub-period

            x0 = [Next_0,Nint_0,m_0]
            # t = np.linspace(hour,hour+Period*24) # every time we solve ODE for 24 hours * Period
            t = np.arange(hour, hour+Period*24) # can also ask for report on round hours

            args = (Nintmax,Nintmin,Vmax,Ks,dNextoutdt,dNextindt,miu,dmoutdt,Nintcrit,Z,KI,K0,Ka,
                                                   losses20,teta,umol_to_percent_DW,I0,Temp)

            x = odeint(controlled_N_constST,x0,t,args=args,printmessg=0,hmax=.1)

            NEXT.append(x[: , 0])
            NINT.append(x[: , 1])
            M.append(x[: , 2])
            TT.append(t)

            t_model = np.hstack(TT)
            Next_model = np.hstack(NEXT)
            Nint_model = np.hstack(NINT)
            m_model = np.hstack(M)

        TModAll.append(TT)
        mExpAllTimes.append(mTimeTemp)
        NintExpAllTimes.append(NintTimeTemp)
        NextExpAllTimes.append(NextTimeTemp)

        mModAll.append(m_model)
        NintModAll.append(Nint_model)
        NextModAll.append(Next_model)

        if plot_on is True:
            if len(mTimeTemp_spor) > 0: #np.empty(mTimeTemp_spor) != True:
                mExpAllTimes_spor.append(mTimeTemp_spor)
                NintExpAllTimes_spor.append(NintTimeTemp_spor)
                NextExpAllTimes_spor.append(NextTimeTemp_spor)
        
                # error bars according to calibration error
            yerrm1,yerrNext1,yerrNint1 = [],[],[]
            for i in m_model:
                yerrm1.append(0.202*i)
            for i in Nint_model:
                yerrNint1.append(0.202*i)
            for i in Next_model:
                yerrNext1.append(0.165*i)
        
            plot_result_extra(t_model,Next_model,Nint_model,m_model,Nint=NintTemp_cal,yerrNint=yerrNint1,tNint=NintTimeTemp_cal,m=mTemp_cal,yerrm=yerrm1,tm=mTimeTemp_cal,Next=NextTemp_cal,yerrNext=yerrNext1,tNext=NextTimeTemp_cal,Next_val = NextTemp_val,tNext_val =NextTimeTemp_val,Nint_val = NintTemp_val,tNint_val =NintTimeTemp_val,m_val = mTemp_val,tm_val =mTimeTemp_val,Next_spor = NextTemp_spor,tNext_spor =NextTimeTemp_spor,Nint_spor = NintTemp_spor,tNint_spor=NintTimeTemp_spor, m_spor=mTemp_spor, tm_spor=mTimeTemp_spor)

        
    # Adjusts measurment time to model durations - so that maximum biomass measurements are compared to maximum biomass
    # predictions and not to the initial stocking density (m0)

    TModTemp = []
    TModAllOrg = []
    mModReducedAll, NintModReducedAll, NextModReducedAll = [],[],[]
    for j in range(len(TModAll)): # loops over 5 treatments
        #print(j)
        for k in range(len(TModAll[j])): # Loops over periods in each treatment
            Ttemp = TModAll[j][k]
            for l in Ttemp: #The model ends at 504 hours, but some measurement reach also 50 hours. This sets a 504 hour limit
                if l > 504:
                    l = 504
                TModTemp.append(l)
        TModAllOrg.append(TModTemp)
        TModTemp = []

    for j in range(len(TModAll)):
        gm = interpolate.interp1d(TModAllOrg[j], mModAll[j],kind = 'linear')
        mModReduced = []
        for k in mExpAllTimes[j]:
            #print(mExpAllTimes[i])
            if k >= 168 and k < 180:
                k = 167.9
            elif k >= 336 and k < 345:
                k = 335.9
            elif k > 504:
                k = 504
            #print(j)
            mModReduced.append(gm(k-1))

        mModReducedAll.append(mModReduced)
        gNint = interpolate.interp1d(TModAllOrg[j], NintModAll[j],kind = 'linear')
        NintModReduced = []
        for k in NintExpAllTimes[j]:
            if k >= 168 and k < 180:
                k = 167.9
            if k >= 336 and k < 345:
                k = 335.9
            if k > 504:
                k = 504
            NintModReduced.append(gNint(k-1))
        NintModReducedAll.append(NintModReduced)
        gNext = interpolate.interp1d(TModAllOrg[j], NextModAll[j],kind = 'linear')    
        NextModReduced = []
        for k in NextExpAllTimes[j]:
            if k >= 168 and k < 180:
                k = 167.9
            if k >= 336 and k < 345:
                k = 335.9
            if k > 504:
                k = 504
            NextModReduced.append(gNext(k-1))
        NextModReducedAll.append(NextModReduced)

    
    # calculate errors
    mSRE_All,NintSRE_All,NextSRE_All = [],[],[]
    for j in range(len(Treatments)):
        mSRE,NintSRE,NextSRE = [],[],[]
        RMSREmAll,RMSRENintAll = [],[]
        
        # df2m = df2_ReducedTemp[(np.isnan(df2_ReducedTemp.DW) != True)&(df2_ReducedTemp.Treatment == Treatments[j])]
        tmp = df2_ReducedTemp[~np.isnan(df2_ReducedTemp.DW)]
        df2m = tmp.loc[tmp.Treatment == Treatments[j]]

        #print('Treatment: ' + str(Treatments[j]) + '\n\nm\n')
        #print('Number of samples: ' + str(len(df2m.Sample)) + '\n')
        l = 0
        for k in df2m.Sample:
            mexp = df2m.DW
            #print(j)
            #print(len(mModReducedAll))
            mmod = mModReducedAll[j]
            mSRE.append(((mexp.iloc[l]-mmod[l])/mmod[l])**2)
            mSRE_All.append(((mexp.iloc[l]-mmod[l])/mmod[l])**2)
            #print('Sample #' + str(math.floor(float(k))))
            #print('Measured biomass: ' + str(round(mexp.iloc[l],3)))
            #print('Modeled biomass: ' + str(round(float(mmod[l]),3)) + '\n')
            l = l + 1
            RMSREm = np.sqrt(np.mean(mSRE))
            
        #print('The RMSRE of m in treamtment ' + str(Treatments[j]) + ' is: ' + str(RMSREm) + '\n')


        # df2Nint = df2_ReducedTemp[(np.isnan(df2_ReducedTemp.N) != True)][(df2_ReducedTemp.Treatment == Treatments[j])]
        tmp = df2_ReducedTemp[~np.isnan(df2_ReducedTemp.N)]
        df2Nint = tmp.loc[tmp.Treatment == Treatments[j]]
        #print('\nNint\n')
        #print('Number of samples: ' + str(len(df2Nint.Sample)) + '\n')
        l = 0
        for k in df2Nint.Sample:
            Nintexp = df2Nint.N
            Nintmod = NintModReducedAll[j]
            NintSRE.append(((Nintexp.iloc[l]-Nintmod[l])/Nintmod[l])**2)
            NintSRE_All.append(((Nintexp.iloc[l]-Nintmod[l])/Nintmod[l])**2)
            #print('Sample #' + str(math.floor(float(k))))
            #print('Measured Nint: ' + str(round(float(Nintexp.iloc[l]),3)))     
            #print('Modeled Nint: ' + str(round(float(Nintmod[l]),3)) + '\n')        
            l = l + 1
            RMSRENint = np.sqrt(np.mean(NintSRE))
            
        #print('\nThe RMSRE of Nint in treamtment ' + str(Treatments[j]) + ' is: ' + str(RMSRENint) + '\n')

        # df2Next = df2_ReducedTemp[(np.isnan(df2_ReducedTemp.NH4) != True)][(df2_ReducedTemp.Treatment == Treatments[j])]
        tmp = df2_ReducedTemp[~np.isnan(df2_ReducedTemp.NH4)]
        df2Next = tmp.loc[tmp.Treatment == Treatments[j]]        
        #print('\nNext\n')
        #print('Number of samples: ' + str(len(df2Next.Sample)) + '\n')
        l = 0
        for k in df2Next.Sample:
            Nextexp = df2Next.NH4
            if Nextexp.iloc[l] < 0:
                Nextexp.iloc[l] = 0
            Nextmod = NextModReducedAll[j]
            NextSRE.append(((Nextexp.iloc[l]-Nextmod[l])/Nextmod[l])**2)
            #print('Sample #' + str(math.floor(float(k))))
            #print('Measured Next: ' + str(round(float(Nextexp.iloc[l]),3)))     
            #print('Modeled Next: ' + str(round(float(Nextmod[l]),3)) + '\n')        
            l = l + 1
            RMSRENext = np.mean(NextSRE)**0.5
            NextSRE_All.append(NextSRE)
        #print('\nThe RMSRE of Next in treamtment ' + str(Treatments[j]) + ' is: ' + str(RMSRENext) + '\n')
    #print('End of treatment ' + str(Treatments[j]) + '\n')
    

    #mSRE_AllTemp = []
    #for i in range(len(mSRE_All)):
    #    for j in range(len(mSRE_All[i])):
    #        mSRE_AllTemp.append(mSRE_All[i][j])
    RMSREm = np.sqrt(np.mean(mSRE_All))

    #NintSRE_AllTemp = []
    #for i in range(len(NintSRE_All)):
    #    for j in range(len(NintSRE_All[i])):
    #        NintSRE_AllTemp.append(NintSRE_All[i][j])
    RMSRENint = np.sqrt(np.mean(NintSRE_All))


    return np.sqrt(RMSREm**2 + RMSRENint**2)
    
    # evaluate_model1.append(RMSREm)
    # evaluate_model2.append(RMSRENint)
    # #evaluate_model3.append(RMSRENext)

    # Y1[p] = round(evaluate_model1[-1],3)
    # Y2[p] = round(evaluate_model2[-1],3)
    # print(r)
    # r = r+1

In [ ]:
from scipy.optimize import minimize

param_values = np.array(problem['bounds']).T.squeeze()
param_values[0,:]


sol = minimize(residual,param_values[0,:],bounds=bounds,method='SLSQP')

In [ ]:
print(sol)

In [ ]:
X = sol.x

In [ ]:
np.savetxt('indoor_calibration.txt',X)

In [ ]:
# This is the main result 
miu = 0.03
lossess20 = X[0]
Nintmax = X[1]
Nintcrit = X[2]
dNextoutdt = X[3]
Ks = X[4]
Vmax = X[5]
KI = X[6]
K0 = X[7]
Ka = X[8]

In [ ]:
residual(X, plot_on = True)